<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/analysis/risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 실업률
- Core CPI
- PMI
- PPI
- IC (실업수당청구건수)


- vix
- CDS


- TED spread
- 10Y - 2Y spread
- 10Y - 3M spread
- high yield spread